In [ ]:
!wget -nc -O dk_2016_raw.csv http://tastselv.ferv.fvm.dk/DFFETILSKUD/f?p=1002:1:8713342471096590:FLOW_EXCEL_OUTPUT_R1193370060391043250_da

In [ ]:
import pandas as pd

In [ ]:
YEAR = 2016

In [ ]:
def get_year(year):
    df = pd.read_csv('dk_%s_raw.csv.gz' % year, encoding='latin-1', delimiter=';', compression='gzip')
    df['year'] = year
    return df

df = get_year(YEAR)
df.head()

In [ ]:
df['year'].value_counts()

In [ ]:
import re
from slugify import slugify

NUM_RE = re.compile('\[Anonym - (\d+)\]')
get_id = lambda x: NUM_RE.match(x).group(1)
df['recipient_id'] = df.apply(lambda x: 'DK-%s' % get_id(x['Navn']) if x['Navn'].startswith('[Anonym - ') else
                                        'DK-%s-%s' % (x['Postnr'], slugify(x['Navn'])), 1)
df['Navn'] = df['Navn'].apply(lambda x: '' if x.startswith('[Anonym - ') else x)
df.head()

In [ ]:
df['recipient_location'] = df.apply(lambda x: '%s, %s' % (x['Kommune'], x['Region']), 1)

df = df.rename(columns={
    'Navn': 'recipient_name',
    'Postnr': 'recipient_postcode',
    'Measure': 'scheme',
    'EU(DKK)': 'amount'
})

df['currency'] = 'DKK'
df['country'] = 'DK'
df['year'] = YEAR
df.head()

In [ ]:
df = df.drop(['Kommune', 'Region', 'National(DKK)', 'Sum(DKK)', 'Totalt(DKK)', 'Unnamed: 8'], 1)
df.head()

In [ ]:
len(df)

In [ ]:
df['amount'] = pd.to_numeric(df['amount'].str.replace('.', '').str.replace(',', '.'))

In [ ]:
df['amount'].sum()

In [ ]:
df.to_csv('dk_%s.csv.gz' % YEAR, index=False, encoding='utf-8', compression='gzip')